# Transactions in SQL

A transaction is a sequence of one or more SQL operations treated as a single unit of work.  Think of it as an "all-or-nothing" operation.

## ACID Properties

Transactions should adhere to the ACID properties to ensure data integrity:

*   **Atomicity:**  All operations within the transaction are treated as a single, indivisible unit.  Either *all* operations succeed, or *none* of them do.  If any operation fails, the entire transaction is rolled back, leaving the database in its original state.

*   **Consistency:** The transaction maintains the integrity of the database.  It ensures that the data remains valid and consistent before and after the transaction is executed.  This typically involves enforcing constraints, rules, and data types.

*   **Isolation:** Concurrent transactions should not interfere with each other. Each transaction should appear to execute as if it were the only transaction running on the database at that time. This prevents data corruption and inconsistencies due to overlapping updates.  Different isolation levels exist to trade off isolation with performance.

*   **Durability:** Once a transaction is committed (successfully completed), the changes made by the transaction are permanent and will survive even system failures (e.g., power outages, crashes).  This is typically achieved by writing the changes to a persistent storage medium (like a hard drive) and using transaction logs.

## Transaction Control Statements

SQL provides the following commands to manage transactions:

*   **`BEGIN TRANSACTION` (or `START TRANSACTION`):**  This statement marks the beginning of a new transaction. All subsequent SQL statements will be considered part of this transaction until a `COMMIT` or `ROLLBACK` is issued.

*   **`COMMIT`:**  This statement saves all the changes made during the transaction to the database.  The transaction is considered successful, and the changes are made permanent.

*   **`ROLLBACK`:**  This statement undoes all the changes made during the transaction. It reverts the database to the state it was in before the transaction began. This is used to handle errors or unexpected conditions that prevent the transaction from completing successfully.

In [ ]:
from sqlalchemy import create_engine, text , exc
from tabulate import tabulate

In [2]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [3]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [4]:
engine = create_engine(DATABASE_URL)

In [5]:
def execute_sql(sql_statement):
    try:
        with engine.connect() as connection:
            connection.execute(text(sql_statement))
            connection.commit()
            print(f"SQL statement executed successfully:\n{sql_statement}")
    except Exception as e:
        print(f"Error executing SQL statement:\n{sql_statement}\nError: {e}")

In [ ]:
execute_sql("""
    CREATE TABLE Customers (
        CustomerID SERIAL PRIMARY KEY,
        FirstName VARCHAR(255),
        LastName VARCHAR(255)
    );
""")

In [ ]:
execute_sql("""
    CREATE TABLE Orders (
        OrderID SERIAL PRIMARY KEY,
        CustomerID INT REFERENCES Customers(CustomerID),
        OrderDate DATE
    );
""")

# Atomicity
* All operations in the transaction are either fully completed or completely rolled back.

In [12]:
def perform_transaction(customer_firstname, customer_lastname, order_date):
    """
    Demonstrates a transaction that inserts a new customer and an associated order.
    If any error occurs, the transaction is rolled back.
    """
    try:
        with engine.connect() as connection:
            
            trans = connection.begin()

            # 1. Insert the new customer
            result = connection.execute(
                text("""
                    INSERT INTO Customers (FirstName, LastName)
                    VALUES (:firstname, :lastname)
                    RETURNING CustomerID  -- Returns the newly generated ID
                """),
                {"firstname": customer_firstname, "lastname": customer_lastname}
            )
            customer_id = result.scalar()

            # 2. Insert the order for the new customer
            connection.execute(
                text("""
                    INSERT INTO Orders (CustomerID, OrderDate)
                    VALUES (:customer_id, :order_date)
                """),
                {"customer_id": customer_id, "order_date": order_date}
            )

            
            trans.commit()
            print("Transaction committed successfully: Customer and Order inserted.")

    except Exception as e:
    
        if 'trans' in locals() and trans:
            trans.rollback()
        print(f"Transaction rolled back due to error: {e}")

In [11]:
perform_transaction("Invalid", "Customer", "2023-11-23")

Transaction committed successfully: Customer and Order inserted.


### Example to demonstrate rollback

In [10]:
perform_transaction("Invalid", "Customer", "2023-11")

Transaction rolled back due to error: (psycopg2.errors.InvalidDatetimeFormat) invalid input syntax for type date: "2023-11"
LINE 3:                     VALUES (8, '2023-11')
                                       ^

[SQL: 
                    INSERT INTO Orders (CustomerID, OrderDate)
                    VALUES (%(customer_id)s, %(order_date)s)
                ]
[parameters: {'customer_id': 8, 'order_date': '2023-11'}]
(Background on this error at: https://sqlalche.me/e/20/9h9h)


C:\Users\tiled\AppData\Local\Temp\ipykernel_98096\2678385452.py:38: SAWarning: transaction already deassociated from connection
  trans.rollback()


# Isolation
* Concurrent transactions should not interfere with each other.

In [13]:
import threading
import time

In [ ]:
def transaction_a(delay=0):
    """Simulates transaction A"""
    conn = engine.connect()
    trans = None

    try:
        trans = conn.begin()
        
        print("Transaction A: Starting transaction")

        result = conn.execute(text("SELECT FirstName FROM Customers WHERE CustomerID = 1"))
        
        initial_name = result.scalar()
        
        print(f"Transaction A: Initial FirstName = {initial_name}")

        print("Transaction A: Updating FirstName to 'Robert'")
        
        conn.execute(
            text("UPDATE Customers SET FirstName = 'Robert' WHERE CustomerID = 1")
        )

        time.sleep(delay)

        trans.commit()
        print("Transaction A: Transaction committed")

    except Exception as e:
        if trans is not None:
            trans.rollback()
        print(f"Transaction A: Transaction rolled back: {e}")
    finally:
        conn.close()


In [17]:
def transaction_b(delay=0):
    """Simulates transaction B"""
    conn = engine.connect()
    trans = None
    try:
        trans = conn.begin()
        print("Transaction B: Starting transaction")

        result = conn.execute(text("SELECT FirstName FROM Customers WHERE CustomerID = 1"))
        
        initial_name = result.scalar()
        
        print(f"Transaction B: Initial FirstName = {initial_name}")

        print("Transaction B: Updating FirstName to 'Alice'")
        
        conn.execute(
            text("UPDATE Customers SET FirstName = 'Alice' WHERE CustomerID = 1")
        )

        time.sleep(delay)

        trans.commit()
        
        print("Transaction B: Transaction committed")

    except Exception as e:
        if trans is not None:
            trans.rollback()
        print(f"Transaction B: Transaction rolled back: {e}")
    finally:
        conn.close()


In [18]:
def run_transactions(delay=0):
    """Runs two concurrent transactions"""
    
    thread_a = threading.Thread(target=transaction_a, args=(delay,))
    thread_b = threading.Thread(target=transaction_b, args=(delay,))

    thread_a.start()
    thread_b.start()

    thread_a.join()
    thread_b.join()


    conn = engine.connect()
    
    try:
        result = conn.execute(text("SELECT FirstName FROM Customers WHERE CustomerID = 1"))
        final_name = result.scalar()
        print(f"Final FirstName in database: {final_name}")
    finally:
        conn.close()

In [19]:
# Run the transactions with no delay (to see if there is any concurrency issue.)
print("Running concurrent transactions with minimal delay:")
run_transactions()


Running concurrent transactions with minimal delay:
Transaction A: Starting transaction
Transaction A: Initial FirstName = John
Transaction A: Updating FirstName to 'Robert'
Transaction A: Transaction committed
Transaction B: Starting transaction
Transaction B: Initial FirstName = Robert
Transaction B: Updating FirstName to 'Alice'
Transaction B: Transaction committed
Final FirstName in database: Alice


In [20]:
# Running concurrent transactions with high delay (to see the difference if high delay.)
print("\nRunning concurrent transactions with 1 second delay:")
run_transactions(delay=1)


Running concurrent transactions with 1 second delay:
Transaction A: Starting transaction
Transaction B: Starting transaction
Transaction B: Initial FirstName = Alice
Transaction B: Updating FirstName to 'Alice'
Transaction A: Initial FirstName = Alice
Transaction A: Updating FirstName to 'Robert'
Transaction B: Transaction committed
Transaction A: Transaction committed
Final FirstName in database: Robert


# Consistency

* The transaction maintains the integrity of the database
* In this example, we intentionally try to insert an order with a non-existent CustomerID. The test expects an IntegrityError (specifically, a ForeignKeyError within it) to be raised and asserts that this happens.

In [21]:
def test_foreign_key_violation():
    """Tests that inserting an order with an invalid CustomerID raises an exception."""
    try:
        with engine.connect() as connection:
            trans = connection.begin()
            try:
                connection.execute(
                    text("""
                        INSERT INTO Orders (CustomerID, OrderDate)
                        VALUES (:customer_id, :order_date)
                    """),
                    {"customer_id": 999, "order_date": "2023-11-23"} 
                )
                trans.commit()
                
                assert False, "ForeignKeyError was not raised"
                
            except exc.IntegrityError as e:
                print(f"Expected ForeignKeyError caught: {e}")
                trans.rollback()
            except Exception as e:
                print(e)
                trans.rollback()

    except Exception as e:
        print(e) 

# Durability

* Once a transaction is committed, the changes are permanent.

In [22]:
def test_durability():
    """Simulates a crash after a transaction commit to verify durability."""
    try:
        with engine.connect() as connection:
            trans = connection.begin()
           
            connection.execute(
                text("""
                    INSERT INTO Customers (FirstName, LastName)
                    VALUES ('CrashTest', 'Dummy')
                """)
            )

            
            trans.commit()
            print("Transaction committed successfully.")
            trans = None 

           

            print("Simulating a system crash... (Manually shut down the database server now!)")
            input("Press Enter after you have simulated the crash...")

        with engine.connect() as connection:
            result = connection.execute(
                text("""
                    SELECT * FROM Customers WHERE FirstName = 'CrashTest'
                """)
            )
            customer = result.fetchone()

            if customer:
                print("Durability test passed: Data survived the crash.")
            else:
                print("Durability test failed: Data was lost during the crash.")

    except Exception as e:
        if trans is not None:
            trans.rollback()

        print(f"test_durability error{e}")


In [23]:
test_durability()

Transaction committed successfully.
Simulating a system crash... (Manually shut down the database server now!)
Durability test passed: Data survived the crash.
